In [1]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [ ]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

은닉층(hidden layer)
- 은닉층: 입력층과 출력층 사이에 있는 모든 층(몇 개든 상관 없음)
- 출력층 뉴런 개수보단 많게 두는 것이 일반적!

심층 신경망 만들기
- 저번처럼 밀집층 객체를 만들고, Sequential()로 전달해서 만들 수 있음.
- 근데 이번엔 층이 2개라는 점이 다른 것 + 은닉층 activation은 가장 대표적인 시그모이드로 일단 해봄

In [ ]:
# 이번엔 층을 2개 만들겠음.                                                      역시 첫 번째 층엔 input_shape해줘야 하고
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 유닛 100개자리 은닉층
dense2 = keras.layers.Dense(10, activation='softmax') # 유닛 10개짜리 출력층

In [ ]:
# 만든 층 2개를 쌓아서 '심층 신경망' 모델 만들기
model = keras.Sequential([dense1, dense2]) # 순서대로 넣으면 됨.

In [ ]:
model.summary()

.summary() 로 모델에 대한 유용한 정보를 확인 가능.
- 'Output Shape' : (경사하강법 배치크기, 출력 유닛 개수)를 보여줌. 훈련을 몇 개의 배치로 할지 모르니까, 유연하게 받을 수 있도록 None으로 되어있음.
- 'Param' : 모델 파라미터, 즉 각 층의 가중치&절편의 개수를 보여줌.
    - 1st 층 = 784 * 100 (가중치) + 100(절편) = 78,500
    - 2nd 층 = 100 * 10 (가중치) + 10(절편) = 1,010

심층 신경망 만들기(2)
- 층을 여러 개 추가하는 다른 방법도 있음 = 따로 층 객체 만들어서 전달하지 않고, Dense() 클래스를 아예 Sequential()안에서 바로

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [ ]:
model.summary() # 한 번에 다 쓰니까 편하긴 한데, 층 많아지면 한 문장이 너무 길어져버릴 수 있음

심층신경만 만들기(3)
- 그래서 실제로 젤 많이 쓰는 건 이 세번째 방법 = 모델에 .add()로 원하는 만큼 추가
- 이러면 add() 속에 if문 등을 활용해서 조건에 따라 층을 추가할 수도 있음

In [ ]:
model = keras.Sequential() # 모델 객체는 처음 비워두고
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax')) # add로 추가하는 식

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

활성화 함수
:⭐은닉층에 활성화함수를 적용하는 이유
- 2개의 선형 계산식이 연속된 형태로 있으면 중간 미지수가 사라지듯이, 신경망에서도 연속으로 쌓는 층이 그냥 선형적인 계산으로만 구성되면, 사실상 중간에 낀 은닉층들의 존재 의미가 사라져버리게 됨.
- 그래서 은닉층의 결괏값을 비선형적으로 비틀어주는 계산이 필요함

렐루 함수
- 기존에 많이 쓰던 시그모이드 함수의 단점: 출력값(z)이 너무 크거나 작을 땐 함수겂의 차이가 너무 미미함 -> 변화에 빠르게 대응하지 못하니까 결국 층을 깊게 쌓기가 힘듦
- 그래서 새롭게 제안된 게 바로 렐루 함수
  : z가 0보다 작으면 0으로 출력하고, z가 0보다 크면 그대로 z를 출력함.(이미지 처리에 좋은 성능을 내는 것으로 알려져있음)

심층신경망 만들기(4)
- Flatten(): 입력데이터를 1차원으로 펼쳐주는 유틸리티 층(그냥 편의를 위한 층)
- => 이러면 좋은 게, summary 출력해보면 입력값 개수를 바로 알 수 있음!(걍 전처리 과정을 점점 모델 내에 포함시키는 추세의 연장선이라 보면 될듯)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary() #summary만 봐도, 이 모델의 입력값은 784개의 1차원 배열이구나를 알 수 있다.

In [ ]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

In [ ]:
model.evaluate(val_scaled, val_target)

4. 옵티마이저

신경망의 하이퍼파라미터
- 신경망에는 특히 하이퍼파라미터(모델학습x 지정 필요o)가 많음.
- ex) 은닉층 뉴런 개수 100, 활성화함수 activation, 층의 종류 Dense Flatten, 배치크기 batch_size, 반복횟수 epochs 등이 있었음.

옵티마이저 설정
- 옵티마이저도 하이퍼파라미터 중 하나로, 경사하강법의 종류에 대한 설정임.
- ex) SGD() : 이름은 그냥 SGD인데 실제로는 미니배치 경사하강법 (기본 배치크기=32)

In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

In [ ]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

In [ ]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

In [ ]:
model.evaluate(val_scaled, val_target)

심층신경망 → 이번엔 입력층과 출력층 사이에 은닉층이라는 걸 하나 추가함! → 층 추가하는 방법 (1) 객체로 만들고 전달 (2) Sequential 내에서 바로 생성 (3) 객체는 아니고 add 메소드로 전달 → 시그모이드 활성화함수의 단점을 극복한 렐루 활성화함수를 새롭게 도입 + 입력데이터를 일렬로 펼쳐주는 Flatten 층도 새로 도입 → 확실히 성능 좋아짐! → 더 나아가, 하강법의 종류를 설정해서 성능 더더욱 높일 수 있는 옵티마이저까지 배움!